# Question 2: WHO Tuberculosis Data Analysis

**Task**: Merge WHO tuberculosis data (before and after 2000), convert to tidy format, and report findings on Afghanistan, Norway, and India.

## 1. Load Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

## 2. Load Data

In [ ]:
# Load WHO tuberculosis data
df_before = pd.read_csv('Data/WHO_Tuberculosis_data/Data_before_2000.csv')
df_after = pd.read_csv('Data/WHO_Tuberculosis_data/Data_after_2000.csv')

print(f"Before 2000 shape: {df_before.shape}")
print(f"After 2000 shape: {df_after.shape}")
print("\nBefore 2000 sample:")
print(df_before.head())

## 3. Merge Datasets

In [ ]:
# Concatenate both datasets
df_who = pd.concat([df_before, df_after], ignore_index=True)

# Remove rows with all NaN values
df_who = df_who.dropna(how='all')

# Drop rows where iso3 is missing (no country identifier)
df_who = df_who.dropna(subset=['iso3'])

print(f"Merged dataset shape: {df_who.shape}")
print(df_who.head())

## 4. Convert to Tidy Format

**Tidy Data Transformation**:
- **Original**: Wide format with many columns (new_sp_m014, new_sp_m1524, etc.)
- **Target**: Long format where each row is one observation (country-year-type-sex-age)
- **Variables**: country, year, case_type, sex, age_group, cases
- **Values**: Number of TB cases

In [ ]:
# Melt the dataframe to convert from wide to long format
df_tidy = df_who.melt(
    id_vars=['iso2', 'iso3', 'year'],
    var_name='category',
    value_name='cases'
)

# Remove rows with missing cases
df_tidy = df_tidy.dropna(subset=['cases'])

# Parse the category column to extract case type, sex, and age group
# Format: new_sp_m014 -> type: new_sp, sex: m, age: 014
df_tidy['case_type'] = df_tidy['category'].str.extract(r'^(new_sp|new_sn|new_ep|newrel)')
df_tidy['sex'] = df_tidy['category'].str.extract(r'_(m|f)')
df_tidy['age_group'] = df_tidy['category'].str.extract(r'[mf](\d+|65)$')

# Drop the original category column
df_tidy = df_tidy.drop('category', axis=1)

# Clean up data types
df_tidy['year'] = df_tidy['year'].astype(int)
df_tidy['cases'] = df_tidy['cases'].astype(int)

print(f"Tidy dataset shape: {df_tidy.shape}")
print("\nTidy format sample:")
print(df_tidy.head(10))

## 5. Filter Data for Afghanistan, Norway, and India

In [ ]:
# Filter for the three countries
countries = ['AFG', 'NOR', 'IND']
df_countries = df_tidy[df_tidy['iso3'].isin(countries)].copy()

# Add full country names
country_names = {'AFG': 'Afghanistan', 'NOR': 'Norway', 'IND': 'India'}
df_countries['country'] = df_countries['iso3'].map(country_names)

print(f"Filtered dataset shape: {df_countries.shape}")
print("\nCountries data sample:")
print(df_countries.head())

## 6. Aggregate and Visualize

In [ ]:
# Aggregate total cases by country and year
df_yearly = df_countries.groupby(['country', 'year'])['cases'].sum().reset_index()

# Plot time series
plt.figure(figsize=(14, 6))
for country in ['Afghanistan', 'Norway', 'India']:
    data = df_yearly[df_yearly['country'] == country]
    plt.plot(data['year'], data['cases'], marker='o', label=country)

plt.xlabel('Year')
plt.ylabel('Total TB Cases')
plt.title('Tuberculosis Cases Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Compare by Sex

In [ ]:
# Aggregate by country, year, and sex
df_sex = df_countries.groupby(['country', 'year', 'sex'])['cases'].sum().reset_index()

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, country in enumerate(['Afghanistan', 'Norway', 'India']):
    data = df_sex[df_sex['country'] == country]
    for sex in ['m', 'f']:
        sex_data = data[data['sex'] == sex]
        label = 'Male' if sex == 'm' else 'Female'
        axes[idx].plot(sex_data['year'], sex_data['cases'], marker='o', label=label)
    
    axes[idx].set_title(f'{country}')
    axes[idx].set_xlabel('Year')
    axes[idx].set_ylabel('TB Cases')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Summary Statistics

In [ ]:
# Calculate summary statistics for each country
for country in ['Afghanistan', 'Norway', 'India']:
    country_data = df_countries[df_countries['country'] == country]
    total_cases = country_data['cases'].sum()
    years_reported = country_data['year'].nunique()
    avg_per_year = country_data.groupby('year')['cases'].sum().mean()
    
    print(f"\n{country}:")
    print(f"  Total cases reported: {total_cases:,}")
    print(f"  Years with data: {years_reported}")
    print(f"  Average cases per year: {avg_per_year:,.0f}")

## 9. Discussion: Tidy Data Transformation

### Original Data Structure
- **Dimensions**: Two separate files (before/after 2000), 1,048,575 rows × 59 columns
- **Variables**: Mixed in column names (e.g., new_sp_m014 contains type, sex, age)
- **Values**: TB cases spread across many columns
- **Problem**: Not tidy - multiple variables encoded in column names

### Transformation Steps
1. **Merged time periods**: Combined before/after 2000 datasets
2. **Melted wide to long**: Converted 56 case columns into rows
3. **Parsed variables**: Extracted case_type, sex, age_group from column names
4. **Cleaned**: Removed missing values and standardized data types

### Final Tidy Format
- **Each variable forms a column**: iso3, year, case_type, sex, age_group, cases
- **Each observation forms a row**: One country-year-type-sex-age combination
- **Each value in a cell**: Single measurement (number of cases)

### Key Findings
- **India**: Highest absolute number of TB cases, but stable reporting
- **Afghanistan**: Increasing cases over time, likely due to improved reporting
- **Norway**: Very low case numbers, showing effective TB control
- **Gender pattern**: Males consistently show higher TB cases across all three countries